# qBraid QIR simulator batch job submission

In [ ]:
from qbraid.runtime import QbraidProvider
from qbraid.visualization import plot_histogram
from qbraid.programs import QPROGRAM_REGISTRY

In [ ]:
provider = QbraidProvider()

In [ ]:
device = provider.get_device("qbraid_qir_simulator")

device.status()

In [ ]:
target_spec = device.profile.get("program_spec")

target_spec

In [ ]:
QPROGRAM_REGISTRY

In [ ]:
graph = device.scheme.conversion_graph

graph.plot(legend=True)

Create batch of GHZ state circuits using OpenQASM 3 and Cirq

In [ ]:
import cirq
import qiskit

from qbraid.visualization import circuit_drawer


def qasm3_ghz() -> str:
    return """
OPENQASM 3.0;
bit[3] b;
qubit[3] q;
h q[0];
cx q[0], q[1];
cx q[1], q[2];
b[0] = measure q[0];
b[1] = measure q[1];
b[2] = measure q[2];
    """


def cirq_ghz() -> cirq.Circuit:
    """Returns Cirq GHZ circuit"""
    q0, q1, q2 = cirq.LineQubit.range(3)
    circuit = cirq.Circuit(
        cirq.ops.H(q0),
        cirq.ops.CNOT(q0, q1),
        cirq.ops.CNOT(q1, q2),
        cirq.measure((q0, q1, q2), key="result"),
    )
    return circuit


circuits = [qasm3_ghz(), cirq_ghz()]

circuit_drawer(circuits[0])

Submit quantum jobs to remote qBraid QIR simulator and plot results

In [ ]:
batch_jobs = device.run(circuits, shots=100)

In [ ]:
for job in batch_jobs:
    print(job.status())

In [ ]:
batch_results = [job.result() for job in batch_jobs]

batch_counts = [result.measurement_counts() for result in batch_results]

In [ ]:
plot_histogram(batch_counts)